# General code for road network

In [1]:
import geopandas as gpd
import folium
import json

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [3]:
# Functions: Getting RWS network

def DutchRDtoWGS84(rdX, rdY):
    """ Convert DutchRD to WGS84
    """
    RD_MINIMUM_X = 11000
    RD_MAXIMUM_X = 280000
    RD_MINIMUM_Y = 300000
    RD_MAXIMUM_Y = 630000
    if (rdX < RD_MINIMUM_X or rdX > RD_MAXIMUM_X
        or rdY < RD_MINIMUM_Y or rdY > RD_MAXIMUM_Y):
        resultNorth = -1
        resultEast = -1
        return resultNorth, resultEast
    # else
    dX = (rdX - 155000.0) / 100000.0
    dY = (rdY - 463000.0) / 100000.0
    k = [[3600 * 52.15517440, 3235.65389, -0.24750, -0.06550, 0.0],
        [-0.00738   ,   -0.00012,  0.0    ,  0.0    , 0.0],
        [-32.58297   ,   -0.84978, -0.01709, -0.00039, 0.0],
        [0.0       ,    0.0    ,  0.0    ,  0.0    , 0.0],
        [0.00530   ,    0.00033,  0.0    ,  0.0    , 0.0],
        [0.0       ,    0.0    ,  0.0    ,  0.0    , 0.0]]
    l = [[3600 * 5.38720621,    0.01199,  0.00022,  0.0    , 0.0],
        [5260.52916   ,  105.94684,  2.45656,  0.05594, 0.00128],
        [-0.00022   ,    0.0    ,  0.0    ,  0.0    , 0.0],
        [-0.81885   ,   -0.05607, -0.00256,  0.0    , 0.0],
        [0.0       ,    0.0    ,  0.0    ,  0.0    , 0.0],
        [0.00026   ,    0.0    ,  0.0    ,  0.0    , 0.0]]
    resultNorth = 0
    resultEast = 0
    powX = 1

    for p in range(6):
        powY = 1
        for q in range(5):
            resultNorth = resultNorth + k[p][q] * powX * powY / 3600.0
            resultEast = resultEast + l[p][q] * powX * powY / 3600.0
            powY = powY * dY
        powX = powX * dX
    return resultNorth, resultEast

def WGS84toDutchRD(wgs84East, wgs84North):
    # translated from Peter Knoppers's code

    # wgs84East: longtitude
    # wgs84North: latitude

    # Western boundary of the Dutch RD system. */
    WGS84_WEST_LIMIT = 3.2

    # Eastern boundary of the Dutch RD system. */
    WGS84_EAST_LIMIT = 7.3

    # Northern boundary of the Dutch RD system. */
    WGS84_SOUTH_LIMIT = 50.6

    # Southern boundary of the Dutch RD system. */
    WGS84_NORTH_LIMIT = 53.7

    if (wgs84North > WGS84_NORTH_LIMIT) or \
        (wgs84North < WGS84_SOUTH_LIMIT) or \
        (wgs84East < WGS84_WEST_LIMIT) or \
        (wgs84East > WGS84_EAST_LIMIT):
        resultX = -1
        resultY = -1
    else:
        r = [[155000.00, 190094.945,   -0.008, -32.391, 0.0],
            [-0.705, -11832.228,    0.0  ,   0.608, 0.0],
            [0.0  ,   -114.221,    0.0  ,   0.148, 0.0],
            [0.0  ,     -2.340,    0.0  ,   0.0  , 0.0],
            [0.0  ,      0.0  ,    0.0  ,   0.0  , 0.0]]
        s = [[463000.00 ,      0.433, 3638.893,   0.0  ,  0.092],
            [309056.544,     -0.032, -157.984,   0.0  , -0.054],
            [73.077,      0.0  ,   -6.439,   0.0  ,  0.0],
            [59.788,      0.0  ,    0.0  ,   0.0  ,  0.0],
            [0.0  ,      0.0  ,    0.0  ,   0.0  ,  0.0]]
        resultX = 0
        resultY = 0
        powNorth = 1
        dNorth = 0.36 * (wgs84North - 52.15517440)
        dEast = 0.36 * (wgs84East - 5.38720621)

        for p in range(5):
            powEast = 1
            for q in range(5):
                resultX = resultX + r[p][q] * powEast * powNorth
                resultY = resultY + s[p][q] * powEast * powNorth
                powEast = powEast * dEast
            powNorth = powNorth * dNorth
    return resultX, resultY

def calc_distance(line_wkt):
    line = ogr.CreateGeometryFromWkt(line_wkt)
    points = line.GetPoints()
    d = 0
    for p0, p1 in zip(points, points[1:]):
        d = d + geodesic(p0, p1).m
    return d

if __name__=="__main__":
    x, y = WGS84toDutchRD(4.33, 52.04)

In [4]:
#Extract subnetwork
highway_shapefile = 'Snelheid_Wegvakken.shp'
network_temp = gpd.read_file(highway_shapefile)

# Method: Clustering

In [5]:
from pyproj import Transformer
from matplotlib.colors import ListedColormap

In [6]:
inc = pd.read_csv('DataClustering.csv', sep='\t')

In [7]:
customcmap = ListedColormap(["lightcoral", "firebrick", "darkred", "red", "coral", "orangered", "crimson", "sandybrown", "peru", "darkorange", "navajowhite", "moccasin", "orange", "darkgoldenrod", "gold", "darkkhaki", "olive", "yellow", "yellowgreen", "lawngreen", "darkseagreen", "palegreen", "green", "darkgreen", "springgreen", "aquamarine", "turquoise", "lightseagreen", "paleturquoise", "teal", "cyan", "cadetblue", "deepskyblue", "steelblue", "dodgerblue", "slategray", "midnightblue", "blue", "mediumslateblue", "mediumpurple", "blueviolet", "plum", "violet", "purple", "magenta", "deeppink", "pink", "mediumblue", "darkmagenta", "palevioletred"])

### 1. Clustering functions based on centroids

In [8]:
def initiate_medoids(n, list_inc):
    u0 = random.sample(list(inc_long_lat['primaire_locatie_breedtegraad']), n)

    u00 = np.zeros((n, 3))

    for i in range(len(u0)):
        u00[i, 1] = u0[i]
        medoid0 = inc_long_lat['primaire_locatie_breedtegraad'] == u0[i]
        index_nb = medoid0.idxmax()
        u00[i, 0] = index_nb
        u00[i, 2] = inc_long_lat['primaire_locatie_lengtegraad'][index_nb]

    u000 = pd.DataFrame(u00[:, 1:], index=u00[:, 0], columns=['primaire_locatie_breedtegraad', 'primaire_locatie_lengtegraad'])
    return u000

In [9]:
def calc_err(xy1, xy2):
    return np.sqrt(np.sum((xy1 - xy2)**2))

In [10]:
def medoid_assignation(list_inc, u, n=316):
    medoid_near_inc = np.zeros(len(list_inc))

    for i in range(len(list_inc)):
        e = np.zeros(n)
        for c in range(n):
            e[c] = calc_err(u.iloc[c, :], list_inc.iloc[i,:])
        
        medoid_near_inc[i] = np.where(e==np.amin(e))[0].tolist()[0]

    return medoid_near_inc

In [11]:
def medoid_error(list_inc, u, n=316):
    medoid_error = np.zeros(len(list_inc))

    for i in range(len(list_inc)):
        e = np.zeros(n)
        for c in range(n):
            e[c] = calc_err(u.iloc[c, :], list_inc.iloc[i,:])
        medoid_error[i] = np.amin(e)
    return medoid_error

In [12]:
def medoids_iterations(medoids_values, list_inc, n=316):
    list_index_medoids = np.zeros(n)
    for m in range(n):
        temp_list = np.zeros(len(list_inc['primaire_locatie_breedtegraad']))
        for k in range(len(list_inc['primaire_locatie_breedtegraad'])):
            temp_list[k] = np.sqrt((medoids_values['primaire_locatie_breedtegraad'][m] - list_inc['primaire_locatie_breedtegraad'][k])**2 + (medoids_values['primaire_locatie_lengtegraad'][m] - list_inc['primaire_locatie_lengtegraad'][k])**2)
        list_index_medoids[m] = np.argmin(temp_list)

    list_medoids = np.zeros((n, 2))
    for i, j in zip(range(n), list_index_medoids):
        list_medoids[i, 0] = list_inc['primaire_locatie_breedtegraad'][j]
        list_medoids[i, 1] = list_inc['primaire_locatie_lengtegraad'][j]
        
    medoids_values = pd.DataFrame(list_medoids, columns=['primaire_locatie_breedtegraad', 'primaire_locatie_lengtegraad'])
    return medoids_values

In [23]:
def clustering(list_inc, n, tol=20):
    new_list_inc = list_inc.copy()
    iterations_error = []
    continue_yn = True
    iteration = 0
    medoids = initiate_medoids(n, new_list_inc)
    print('Step 0')

    while(continue_yn):
        # Step 1 - Assign medoids and calculate error
        new_list_inc['medoid'] = medoid_assignation(new_list_inc, medoids, n)
        print(f'Step {iteration}.1')
        
        # Step 2,3 - Update medoids position
        medoids = new_list_inc.groupby('medoid').agg('mean').reset_index(drop = True)
        medoids = medoids_iterations(medoids, new_list_inc, n)
        print(f'Step {iteration}.2')

        u_s_error = medoid_error(new_list_inc, medoids, n)
        iterations_error.append(np.sum(u_s_error))
        print(f'Step {iteration}.3')
        print(iterations_error[iteration])
        
        # Step 4 - Restart the iteration
        if iteration>0:
            print(iterations_error[iteration - 1] - iterations_error[iteration])
            if iterations_error[iteration - 1] - iterations_error[iteration] <= tol:
                continue_yn = False
        iteration+=1
        print(f'Iteration: {iteration}')
        print(f'Shape meoids: {np.shape(medoids)}')
        
    return new_list_inc['medoid'], u_s_error

### 2. Assignment of initial centroids

In [14]:
inc_long_lat = inc[['primaire_locatie_breedtegraad', 'primaire_locatie_lengtegraad']]

In [15]:
n = 158

In [16]:
coords = ['primaire_locatie_breedtegraad', 'primaire_locatie_lengtegraad']

### 3. Clustering with iterations

In [ ]:
inc_long_lat['medoid'], inc_long_lat['error'] =  clustering(inc_long_lat[coords], n)

Step 0


### 4. Save and import files (to prevent running code again)

In [ ]:
inc_long_lat.to_csv('incidents_clustered_medoids.csv')

In [ ]:
xy_c = np.zeros((n, 2))

for xy in range(n):
    xy_c[xy][0] = u1['primaire_locatie_breedtegraad'][xy]
    xy_c[xy][1] = u1['primaire_locatie_lengtegraad'][xy]

In [ ]:
medoids_coords = pd.DataFrame(xy_c, columns=['breedtegraad', 'lengtegraad'])
medoids_coords.to_csv('medoids_coords.csv')

In [ ]:
transformer1 = Transformer.from_crs("EPSG:4326", "EPSG:28992")
rds_xy = np.zeros((n, 2))

t = transformer1.transform(xy_c[:, 0], xy_c[:, 1])
rds_xy[:, 0] = t[0]
rds_xy[:, 1] = t[1]

In [ ]:
medoids_rds = pd.DataFrame(rds_xy, columns=['breedtegraad', 'lengtegraad'])
medoids_rds.to_csv('medoids_rds.csv')

In [ ]:
print('x')

In [ ]:
new_ill = pd.read_csv('incidents_clustered_medoids.csv')
new_cc = pd.read_csv('medoids_coords.csv')
new_rds = pd.read_csv('medoids_rds.csv')

### 5. Plots of clustering based on centroids

#### 5.1. Plot centroids on roadmap

In [ ]:
network_temp.plot(figsize=(12, 10))
plt.plot(new_rds['breedtegraad'], new_rds['lengtegraad'], 'ro');

#### 5.2. Plot incidents and centroids clustered

In [ ]:
plt.figure(figsize=(160, 200))
plt.scatter(new_ill['primaire_locatie_lengtegraad'], new_ill['primaire_locatie_breedtegraad'],  
            marker = 'o', c=new_ill['medoid'].astype('category'), 
            cmap = customcmap, s=150, alpha=0.5)
plt.scatter(new_cc['lengtegraad'], new_cc['breedtegraad'],  
            marker = 's', s=2500, c=np.arange(0, 50, 1), 
            cmap = customcmap)

#### 5.3. Plot cluster sizes

In [ ]:
number_inc_cluster = np.zeros(n)

for i in range(n):
    for j in range(len(new_ill['medoid'])):
        if int(new_ill['medoid'][j]) == i:
            number_inc_cluster[i] += 1
        else:
            number_inc_cluster[i] += 0

In [ ]:
network_temp.plot(figsize=(160, 200), linewidth=5)

plt.scatter(new_rds['breedtegraad'], new_rds['lengtegraad'],  
            marker = 's', s=10*number_inc_cluster, c=np.arange(0, 50, 1), 
            cmap = customcmap)

for t in range(n):
    plt.text(new_rds['breedtegraad'][t] + 1000, new_rds['lengtegraad'][t] + 1000, str(int(number_inc_cluster[t])), size=100)

#### 5.4. Plot average speed of clusters (large approximations)

In [ ]:
v = 80 #km/h
factor = 2.6

In [ ]:
transformer2 = Transformer.from_crs("EPSG:28992", "EPSG:4326")

dist1 = np.zeros((n, 2))
dist2 = np.zeros((len(new_ill['primaire_locatie_breedtegraad']), 2))

t41 = transformer2.transform(new_cc['breedtegraad'], new_cc['lengtegraad'])
t42 = transformer2.transform(new_ill['primaire_locatie_breedtegraad'], new_ill['primaire_locatie_lengtegraad'])

dist1[:, 0] = t41[0]
dist1[:, 1] = t41[1]
dist2[:, 0] = t42[0]
dist2[:, 1] = t42[1]

In [ ]:
tot_time = np.zeros(n)
tot_points = np.zeros(n)
av_time = np.zeros(n)

for i in range(n):
    for j in range(len(new_ill['primaire_locatie_lengtegraad'])):
        if new_ill['medoid'][j] == i:
            absolute_dist = np.sqrt((dist2[:, 1][j] - dist1[:, 1][i])**2 + (dist2[:, 0][j] - dist1[:, 0][i])**2)
            tot_time[i] += absolute_dist * v * factor
            tot_points[i] += 1
        else:
            tot_points[i] += 0
    av_time[i] = tot_time[i] / tot_points[i] * 60 * 1000  # time in minutes

In [ ]:
network_temp.plot(figsize=(160, 200), linewidth=5)

plt.scatter(new_rds['breedtegraad'], new_rds['lengtegraad'],  
            marker = 's', s=10*number_inc_cluster, c=np.arange(0, 50, 1), 
            cmap = customcmap)

for t in range(n):
    plt.text(new_rds['breedtegraad'][t] + 1000, new_rds['lengtegraad'][t] + 1000, f'{av_time[t]:.2f}', size=100)